## Bizarre Attempt of Decision Tree or HMM (Maybe) which just returns Anomaly or Normal

In [3]:
from datetime import datetime, timedelta

def process_log(log, ip_data):
    """
    Processes a single log entry and make decisions based on the decision tree.

    Arguments or Args:
        log (str): The log entry in string format.
        ip_data (list): List of lists containing IP data in the format:
                        [IP Address, Last Accessed DateTime, Incorrect Attempts].

    Returns:
        str: "Anomaly Detected" if an anomaly is found, else "Normal".
    """
    
    # Step 1: Checking if "sshd" is in the log
    if "sshd" not in log:
        return "Normal"

    # Step 2: Checking if "authentication failure" is in the log
    if "authentication failure" not in log:
        return "Normal"

    # Extracting the IP address and time from the log
    try:
        ip_address = log.split("rhost=")[1].split()[0]
        date_str = " ".join(log.split()[:3])
        log_time = datetime.strptime(date_str, "%b %d %H:%M:%S")
    except (IndexError, ValueError):
        return "Normal"  # To handle logs with missing or invalid data, just return Normal

    # Step 3: Checking if the IP address is in the list of lists
    for entry in ip_data:
        
        if entry[0] == ip_address:
            
            # Step 4: Checking if the time difference is within 5 seconds
            if abs((log_time - entry[1]).total_seconds()) <= 5:
                
                # Step 5: Increasing incorrect attempts
                entry[2] += 1
                
            else:
                
                # Updating the time and reset incorrect attempts
                entry[1] = log_time
                entry[2] = 1

            # Step 6: Checking if incorrect attempts exceed 5
            if entry[2] > 5:
                return "Anomaly Detected"
            return "Normal"

    # If IP address is not in the list, adding a new entry
    ip_data.append([ip_address, log_time, 1])
    return "Normal"

logs = [
    "Jun 14 15:16:01 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
    "Jun 14 15:16:02 combo sshd(pam_unix)[19937]: check pass; user unknown",
    "Jun 14 15:16:02 combo sshd(pam_unix)[19937]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
    "Jun 14 15:16:03 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:04 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:04 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.3",
    "Jun 14 15:16:04 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.3",
    "Jun 14 15:16:05 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:05 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:05 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:05 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:06 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
    "Jun 14 15:16:06 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:06 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
    "Jun 14 15:16:06 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
    "Jun 14 15:16:06 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
    "Jun 14 15:16:06 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
]

ip_data = []  # List of lists for storing IP data

for log in logs:
    result = process_log(log, ip_data)
    print(result)

print("Updated IP Data:", ip_data)


Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Anomaly Detected
Normal
Anomaly Detected
Normal
Normal
Anomaly Detected
Anomaly Detected
Updated IP Data: [['218.188.2.4', datetime.datetime(1900, 6, 14, 15, 16, 1), 7], ['218.188.2.2', datetime.datetime(1900, 6, 14, 15, 16, 3), 7], ['218.188.2.3', datetime.datetime(1900, 6, 14, 15, 16, 4), 2]]


## Bizarre Attempt of Decision Tree or HMM (Maybe) which returns Anomaly or Normal alogn with IP Addresses

In [7]:
from datetime import datetime, timedelta

def process_log(log, ip_data):
    """
    Process a single log entry and make decisions based on the decision tree proposed by Indrajit.

    Arguments or Args:
        log (str): The log entry in string format.
        ip_data (list): List of lists containing IP data in the format:
                        [IP Address, Last Accessed DateTime, Incorrect Attempts].

    Returns:
        tuple: (str, str) - Status ("Anomaly Detected" or "Normal") and the IP address.
    """
    # Step 1: Checking if "sshd" is in the log
    if "sshd" not in log:
        return "Normal", None

    # Step 2: Checking if "authentication failure" is in the log
    if "authentication failure" not in log:
        return "Normal", None

    # Extracting the IP address and time from the log
    try:
        ip_address = log.split("rhost=")[1].split()[0]
        
        date_str = " ".join(log.split()[:3])
        
        log_time = datetime.strptime(date_str, "%b %d %H:%M:%S")
        
    except (IndexError, ValueError):
        return "Normal", None  # To handle logs with missing or invalid data, just return Normal for now

    # Step 3: Checking if the IP address is in the list of lists
    for entry in ip_data:
        
        if entry[0] == ip_address:
            
            # Step 4: Checking if the time difference is within 5 seconds
            if abs((log_time - entry[1]).total_seconds()) <= 5:
                
                # Step 5: Increasing incorrect attempts
                entry[2] += 1
                
            else:
                
                # Updating the time and reset incorrect attempts
                entry[1] = log_time
                entry[2] = 1

            # Step 6: Checing if incorrect attempts exceed 5
            if entry[2] > 5:
                return "Anomaly Detected", ip_address
            return "Normal", ip_address

    # If IP address is not in the list, adding a new entry in the ip_data
    ip_data.append([ip_address, log_time, 1])
    return "Normal", ip_address

logs = [
    "Jun 14 15:16:01 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
    "Jun 14 15:16:02 combo sshd(pam_unix)[19937]: check pass; user unknown",
    "Jun 14 15:16:02 combo sshd(pam_unix)[19937]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
    "Jun 14 15:16:03 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:04 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:04 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.3",
    "Jun 14 15:16:04 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.3",
    "Jun 14 15:16:05 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:05 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:05 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:05 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:06 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
    "Jun 14 15:16:06 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.2",
    "Jun 14 15:16:06 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
    "Jun 14 15:16:06 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
    "Jun 14 15:16:06 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
    "Jun 14 15:16:06 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4",
]

ip_data = []  # List of lists for storing IP data

for log in logs:
    result, ip = process_log(log, ip_data)
    print(f"Status: {result}, IP: {ip}")

print("Updated IP Data:", ip_data)

Status: Normal, IP: 218.188.2.4
Status: Normal, IP: None
Status: Normal, IP: 218.188.2.4
Status: Normal, IP: 218.188.2.2
Status: Normal, IP: 218.188.2.2
Status: Normal, IP: 218.188.2.3
Status: Normal, IP: 218.188.2.3
Status: Normal, IP: 218.188.2.2
Status: Normal, IP: 218.188.2.2
Status: Normal, IP: 218.188.2.2
Status: Anomaly Detected, IP: 218.188.2.2
Status: Normal, IP: 218.188.2.4
Status: Anomaly Detected, IP: 218.188.2.2
Status: Normal, IP: 218.188.2.4
Status: Normal, IP: 218.188.2.4
Status: Anomaly Detected, IP: 218.188.2.4
Status: Anomaly Detected, IP: 218.188.2.4
Updated IP Data: [['218.188.2.4', datetime.datetime(1900, 6, 14, 15, 16, 1), 7], ['218.188.2.2', datetime.datetime(1900, 6, 14, 15, 16, 3), 7], ['218.188.2.3', datetime.datetime(1900, 6, 14, 15, 16, 4), 2]]
